In [1]:
import magma as m
import inspect
import logging
logging.basicConfig(level=logging.DEBUG)
import fault
from hwtypes import BitVector as BV

The combinational syntax allows you to use `if/else` statements. 

In [2]:
@m.circuit.combinational
def basic_if(I: m.Bits[2], S: m.Bit) -> m.Bit:
    if S:
        x = I[0]
    else:
        x = I[1]
    return x


class Main(m.Circuit):
    io = m.IO(I0=m.In(m.Bit), I1=m.In(m.Bit), S=m.In(m.Bit), O=m.Out(m.Bit))
    io.O @= basic_if(m.bits([io.I0, io.I1]), io.S)

m.compile("build/basic_if", basic_if.circuit_definition)
with open('.magma/basic_if.py', 'r') as f:
    print(f.read())

import magma as m
from mantle import mux as phi


class basic_if(m.Circuit):
    io = m.IO(I=m.In(m.Bits[2]), S=m.In(m.Bit), O=m.Out(m.Bit))
    x_0 = io.I[0]
    x_1 = io.I[1]
    x_2 = phi([x_1, x_0], io.S)
    __magma_ssa_return_value_0 = x_2
    O = __magma_ssa_return_value_0
    m.wire(O, io.O)



In [3]:
tester = fault.PythonTester(Main)
assert tester(0, 1, 0) == 1
assert tester(0, 1, 1) == 0

In [4]:
tester = fault.Tester(Main)
tester(0, 1, 0).expect(1)
tester.circuit.S = 1
tester.eval()
tester.circuit.O.expect(0)
tester.compile_and_run("verilator")

INFO:root:Running tester...
INFO:root:Success!


You can insert code to instance magma circuits inside combinational.

In [5]:
from mantle import Not

@m.circuit.combinational
def invert(a: m.Bit) -> m.Bit:
    return Not()(a)

print(repr(invert.circuit_definition))

invert = DefineCircuit("invert", "a", In(Bit), "O", Out(Bit))
not_inst0 = not()
wire(invert.a, not_inst0.in)
wire(not_inst0.out, invert.O)
EndCircuit()


In [6]:
tester = fault.PythonTester(invert.circuit_definition)
assert tester(0) == 1
assert tester(1) == 0

In [7]:
tester = fault.Tester(invert.circuit_definition)
tester.circuit.a = 1
tester.eval()
tester.circuit.O.expect(0)
tester.circuit.a = 0
tester.eval()
tester.circuit.O.expect(1)
tester.compile_and_run("verilator")

INFO:root:Running tester...
INFO:root:Success!


In [8]:
@m.circuit.sequential
class Counter:
    def __init__(self):
        self.count : m.UInt[16] = 0

    def __call__(self, inc : m.Bit) -> m.UInt[16]:
        if inc:
            self.count = self.count + 1

        O = self.count
        return O


m.compile("Counter", Counter, output="coreir-verilog")
!coreir -i Counter.json -p instancecount -l commonlib

An instance count of all the primitives
invert | instances in current | instances in children | 
  corebit_not | 1 | 0

Mux2xOutUInt16 | instances in current | instances in children | 

Mux2xOutBit | instances in current | instances in children | 

basic_if | instances in current | instances in children | 

Main | instances in current | instances in children | 

Counter_comb | instances in current | instances in children | 
  coreir_add__width16 | 1 | 0
  coreir_const__width16 | 1 | 0

Counter | instances in current | instances in children | 
  coreir_add__width16 | 0 | 1
  coreir_const__width16 | 0 | 1
  coreir_reg_arst__width16 | 1 | 0

{"top":"global.Counter",
"namespaces":{
  "global":{
    "modules":{
      "Counter":{
        "type":["Record",[
          ["inc","BitIn"],
          ["CLK",["Named","coreir.clkIn"]],
          ["ASYNCRESET",["Named","coreir.arstIn"]],
          ["O",["Array",16,"Bit"]]
        ]],
        "instances":{
          "Counter_comb_inst0":{
            "m

In [9]:
tester = fault.PythonTester(Counter, Counter.CLK)
tester.poke(Counter.inc, True)
tester.eval()
for i in range(4):
    print(tester.peek(Counter.O))
    assert tester.peek(Counter.O) == i + 1
    tester.step(2)
tester.poke(Counter.inc, False)
tester.eval()
for i in range(4):
    print(tester.peek(Counter.O))
    assert tester.peek(Counter.O) == 4
    tester.step(2)

1
2
3
4
4
4
4
4
